<a href="https://colab.research.google.com/github/imbalzy/RecipeQA-FInal-Project-2470/blob/main/pair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"
!cp "/content/gdrive/My Drive/Kaggle/kaggle.json" .

Mounted at /content/gdrive


In [2]:
!kaggle datasets download -d jeromeblanchet/recipeqa-nlp-dataset
!unzip recipeqa-nlp-dataset.zip -d data/ > /dev/null

!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

100% 2.54G/2.55G [00:24<00:00, 48.6MB/s]
100% 2.55G/2.55G [00:24<00:00, 111MB/s] 
--2020-12-08 18:30:42--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-12-08 18:30:42--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-12-08 18:30:43--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 20

In [30]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from preprocessrecipeqa import *

In [7]:
batch_size = 1
(train_iter1, train_iter2, train_iter3, train_iter4), (test_iter1, test_iter2, test_iter3, test_iter4), (val_iter1, val_iter2, val_iter3, val_iter4), embedding_index, word_index = preprocess(batch_size)


In [76]:
from matplotlib import pyplot

In [82]:
for i, label in train_iter1:
  for j in i[0]: print(j)
  # print(i[0]['recipe_id'])
  for k in i[0]['context']:
    print(k['body'])
    print(len(k['images']))
  break

recipe_id
context
choice_list
question
[17861, 23909, 3275, 12204, 3859, 10587, 15623, 25089, 19903, 11172, 911, 22875, 10587, 18154, 24410, 5681, 2375, 1824, 22097, 27586, 12604, 6865, 9034, 5267, 16864, 5662, 9034, 3388, 7159, 3275, 17156, 28078, 9352, 25423, 22612, 8979, 25043, 9034, 10755, 24850, 15992, 2061, 3469, 14250, 29039, 7235, 17268, 27142, 16762, 28294, 19903, 22612, 8979, 2375, 17156, 13959, 25043, 6015, 24540, 25575, 3275, 13074, 25423, 12468]
3
[8236, 1824, 22097, 6345, 1671, 11511, 1461, 15491, 21360, 24751, 23120, 19903, 11172, 911, 10305, 16902, 16265, 17130, 17438, 4911, 9558, 10587, 12180]
2
[21968, 15992, 24540, 6345, 29039, 25081, 11275, 17438, 19903, 25423, 22875, 10587, 18154, 24410, 16265, 2375, 2047, 12604, 24741, 24410, 1461, 26876, 19903, 25423, 541, 13302, 10587, 19903, 13074, 541, 22085, 10587, 12804, 16864, 15549, 19903, 25423, 3966, 19903, 356, 19903, 4911, 2858, 12310, 26032, 2406, 6345, 13074, 12604, 17130, 9034, 26032, 27142, 15491, 27586, 15491, 290

In [58]:
def define_model():
  img_inputs = keras.Input(shape=(224, 224, 3))
  l1 = keras.layers.Conv2D(filters = 64, kernel_size = 5, strides=(2, 2), padding='same')
  x = l1(img_inputs)
  l2 = keras.layers.Conv2D(filters = 64, kernel_size = 5, strides=(2, 2), padding='same')
  x = l2(x)
  l3 = keras.layers.Conv2D(filters = 64, kernel_size = 5, strides=(2, 2), padding='same')
  x = l3(x)
  l4 = keras.layers.Conv2D(filters = 64, kernel_size = 5, strides=(2, 2), padding='same')
  x = l4(x)
  flatten = keras.layers.Flatten()
  x = flatten(x)
  densex = keras.layers.Dense(100,activation='tanh')
  x = densex(x)
  text_inputs = keras.Input(shape = (None,100))
  lstm = tf.keras.layers.LSTM(100)

  densey =tf.keras.layers.Dense(100,activation='tanh')
  y = densey(lstm(text_inputs))
  
  concat = keras.layers.Concatenate()
  fc1 = keras.layers.Dense(200)
  z = fc1(concat([x,y]))
  fc2 = keras.layers.Dense(2)
  outputs = fc2(z)
  model = tf.keras.Model(inputs=[text_inputs,img_inputs], outputs=outputs)

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [59]:
model = define_model()

In [65]:
model.summary()

Model: "functional_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_70 (Conv2D)              (None, 112, 112, 64) 4864        input_21[0][0]                   
__________________________________________________________________________________________________
conv2d_71 (Conv2D)              (None, 56, 56, 64)   102464      conv2d_70[0][0]                  
__________________________________________________________________________________________________
conv2d_72 (Conv2D)              (None, 28, 28, 64)   102464      conv2d_71[0][0]                  
______________________________________________________________________________________

In [66]:
model.predict([np.ones([1,10,100]),np.ones([1,224,224,3])])

array([[0.2887484, 0.1946048]], dtype=float32)